In [1]:
import ffn

# pip install pandas-datareader --upgrade

/home/dheeraj/.local/lib/python3.6/site-packages/ffn/core.py:27: MatplotlibDeprecationWarning: The 'warn' parameter of use() is deprecated since Matplotlib 3.1 and will be removed in 3.3.  If any parameter follows 'warn', they should be pass as keyword, not positionally.
  matplotlib.use('agg', warn=False)


In [2]:
returns = ffn.get('aapl,msft,dkng,nvda,lmnd', start='2010-01-01').to_returns().dropna()
returns.calc_mean_var_weights().as_format('.2%')

aapl    50.29%
msft     0.00%
dkng    19.09%
nvda     9.85%
lmnd    20.77%
dtype: object

In [3]:
from empyrical import alpha_beta
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns
from pypfopt import discrete_allocation
from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices
import matplotlib as pyplot
import numpy as np
import pandas as pd

In [19]:
import csv
def get_tickers(year=2016):
    reader = csv.reader(open("TICKER_LIST_CLEANED.csv"))
    next(reader)
    stocks = [line[0].strip() for line in reader if all(punc not in line[0] for punc in ['.','/']) and (line[7] and int(line[7]) < year)]
    return stocks

In [20]:
tickers = get_tickers(year=2019)
print(tickers[41])
# del tickers[41]
print(tickers[1399])
print(tickers[0:39])

ADUS
MCN
['A', 'AA', 'AAOI', 'AAPL', 'AAT', 'AAU', 'ABBV', 'ABCB', 'ABEV', 'ABG', 'ABR', 'ABTX', 'AC', 'ACA', 'ACAD', 'ACB', 'ACC', 'ACEL', 'ACET', 'ACH', 'ACIA', 'ACIU', 'ACLS', 'ACM', 'ACMR', 'ACN', 'ACOR', 'ACP', 'ACRE', 'ACRS', 'ACRX', 'ACU', 'ACV', 'ADAP', 'ADBE', 'ADIL', 'ADMS', 'ADNT', 'ADS']


In [215]:
prices = ffn.get(tickers[0:51], start = '2016-01-01')

In [190]:
# prices = ffn.get('msft,aapl,amzn,lmnd,nvda,crsp,tsla', start='2016-01-01')
benchmark = ffn.get('spy', start='2016-01-01')


In [191]:
msft = ffn.get('msft', start='2016-01-01')
ax = prices.rebase().plot()

In [192]:
returns = prices.to_returns().dropna()
stats = prices.calc_stats()

stats.display()

/home/dheeraj/.local/lib/python3.6/site-packages/ffn/core.py:2056: RuntimeWarning: divide by zero encountered in true_divide
  res = np.divide(er.mean(), std)
/home/dheeraj/.local/lib/python3.6/site-packages/ffn/core.py:2056: RuntimeWarning: divide by zero encountered in true_divide
  res = np.divide(er.mean(), std)
/home/dheeraj/.local/lib/python3.6/site-packages/ffn/core.py:2056: RuntimeWarning: divide by zero encountered in true_divide
  res = np.divide(er.mean(), std)
/home/dheeraj/.local/lib/python3.6/site-packages/ffn/core.py:2056: RuntimeWarning: divide by zero encountered in true_divide
  res = np.divide(er.mean(), std)
/home/dheeraj/.local/lib/python3.6/site-packages/ffn/core.py:2056: RuntimeWarning: divide by zero encountered in true_divide
  res = np.divide(er.mean(), std)
/home/dheeraj/.local/lib/python3.6/site-packages/ffn/core.py:2056: RuntimeWarning: divide by zero encountered in true_divide
  res = np.divide(er.mean(), std)
/home/dheeraj/.local/lib/python3.6/site-packag

Stat                 a           aa          aacg        aaic        aal         aamc        aame        aaoi        aaon        aap         aapl        aat         aau         aaww        aaxn        ab          abb         abbv        abc         abcb        abeo        abev        abg         abio        abm         abmd        abr         abt         abtx        abus        ac          aca         acad        acam        acamu       acb
-------------------  ----------  ----------  ----------  ----------  ----------  ----------  ----------  ----------  ----------  ----------  ----------  ----------  ----------  ----------  ----------  ----------  ----------  ----------  ----------  ----------  ----------  ----------  ----------  ----------  ----------  ----------  ----------  ----------  ----------  ----------  ----------  ----------  ----------  ----------  ----------  ----------
Start                2019-05-09  2019-05-09  2019-05-09  2019-05-09  2019-05-09  2019-05-09  2019-05-09

In [193]:
alpha, beta = alpha_beta(msft, benchmark)
print(beta)

1.0318531639121846


/home/dheeraj/.local/lib/python3.6/site-packages/empyrical/stats.py:1384: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  returns = returns[:, np.newaxis]
/home/dheeraj/.local/lib/python3.6/site-packages/empyrical/stats.py:1387: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  factor_returns = factor_returns[:, np.newaxis]
/home/dheeraj/.local/lib/python3.6/site-packages/empyrical/stats.py:1294: RuntimeWarning: overflow encountered in power
  out=out


In [194]:
returns = prices.pct_change()
 
# mean daily return and covariance of daily returns
# mean_daily_returns = returns.mean()
# cov_matrix = returns.cov()
 
# # portfolio weights
# weights = np.asarray([0.4,0.2,0.1,0.1,0.1,0.1])
 
# portfolio_return = round(np.sum(mean_daily_returns * weights) * 252,2)
# portfolio_std_dev = round(np.sqrt(np.dot(weights.T,np.dot(cov_matrix, weights))) * np.sqrt(252),2)

# print("Expected annualised return: " + str(portfolio_return))
# print("Volatility: " + str(portfolio_std_dev))

In [211]:
# Expected returns and sample covariance
mu = expected_returns.mean_historical_return(prices)
S = risk_models.sample_cov(prices)

# Optimise portfolio for maximum Sharpe Ratio
ef = EfficientFrontier(mu, S)
raw_weights = ef.max_sharpe()
cleaned_weights = ef.clean_weights()
print(cleaned_weights)
ef.portfolio_performance(verbose=True)


OrderedDict([('a', 0.0), ('aa', 0.0), ('aacg', 0.0), ('aaic', 0.0), ('aal', 0.0), ('aamc', 0.0), ('aame', 0.0), ('aaoi', 0.0), ('aaon', 0.0), ('aap', 0.0), ('aapl', 0.50052), ('aat', 0.0), ('aau', 0.0), ('aaww', 0.0), ('aaxn', 0.0), ('ab', 0.0), ('abb', 0.0), ('abbv', 0.0), ('abc', 0.0), ('abcb', 0.0), ('abeo', 0.0), ('abev', 0.0), ('abg', 0.01042), ('abio', 0.0), ('abm', 0.0), ('abmd', 0.0), ('abr', 0.0), ('abt', 0.0), ('abtx', 0.0), ('abus', 0.0), ('ac', 0.0), ('aca', 0.0), ('acad', 0.04178), ('acam', 0.40552), ('acamu', 0.02086), ('acb', 0.0), ('acbi', 0.0), ('acc', 0.0), ('acco', 0.0), ('acel', 0.0), ('acer', 0.0), ('acet', 0.0), ('acgl', 0.0), ('acglo', 0.0), ('acglp', 0.0), ('ach', 0.0), ('achc', 0.0), ('achv', 0.0), ('acia', 0.02089), ('aciu', 0.0), ('aciw', 0.0)])
Expected annual return: 44.7%
Annual volatility: 21.8%
Sharpe Ratio: 1.96


(0.44734805821921664, 0.21791586573463523, 1.961069042763574)

In [212]:
latest_prices = get_latest_prices(prices)
da = DiscreteAllocation(cleaned_weights, latest_prices, total_portfolio_value=6000)
allocation, leftover = da.lp_portfolio()

print(allocation)
print("Funds remaining: ${:.2f}".format(leftover))

{'aapl': 24, 'acad': 5, 'acam': 222, 'acamu': 10, 'acia': 2}
Funds remaining: $2.54


In [209]:
ef = EfficientFrontier(mu, S, weight_bounds=(-1, 1))
ef.efficient_return(target_return=0.2, market_neutral=True)

OrderedDict([('a', 0.0315015893389542),
             ('aa', -0.0092510905889794),
             ('aacg', -0.0063817376487219),
             ('aaic', -0.0055235677012912),
             ('aal', -0.0162371185431683),
             ('aamc', 0.0012143421988103),
             ('aame', -0.0025862487640527),
             ('aaoi', -0.0083601645867665),
             ('aaon', -0.0170365745519976),
             ('aap', -0.0330210264430172),
             ('aapl', 0.1068110333404698),
             ('aat', -0.0452762597296056),
             ('aau', 0.0018391952747618),
             ('aaww', -0.0087653913958821),
             ('aaxn', 0.0090508193847637),
             ('ab', -0.0182116745145118),
             ('abb', 0.0350227676920341),
             ('abbv', 0.0072254169914589),
             ('abc', 0.0116067353851758),
             ('abcb', 0.0407764456776654),
             ('abeo', -0.0044405015864341),
             ('abev', -0.0265045069387282),
             ('abg', 0.0420824059241794),
            

# Script For Portfolio Optimization Based on Efficient Frontier

In [17]:
# print(type(len(tickers)))
# index = len(tickers)
# columns = tickers

# all_prices = pd.DataFrame(index=index, columns=columns)

all_prices = ffn.get('a', start = '2016-01-01')
# all_prices
tickers[1701]


'OPTT'

In [21]:
for i in range(1,len(tickers)):
    print(i)
    prices = ffn.get(tickers[i], start = '2016-01-01')
    

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


1861
1862
1863
1864
1865
1866
1867
1868
1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060


In [ ]:
prices = ffn.get(tickers, start = '2016-01-01')
benchmark = ffn.get('spy', start='2016-01-01')
prices.to_pickle('prices_data.pkl')

In [23]:
# Expected returns and sample covariance
mu = expected_returns.mean_historical_return(prices)
S = risk_models.sample_cov(prices)

# Optimise portfolio for maximum Sharpe Ratio
ef = EfficientFrontier(mu, S)
raw_weights = ef.max_sharpe()
cleaned_weights = ef.clean_weights()
print(cleaned_weights)
ef.portfolio_performance(verbose=True)


/home/dheeraj/.local/lib/python3.6/site-packages/pypfopt/risk_models.py:69: UserWarning: The covariance matrix is non positive semidefinite. Amending eigenvalues.
  "The covariance matrix is non positive semidefinite. Amending eigenvalues."
/home/dheeraj/.local/lib/python3.6/site-packages/pypfopt/risk_models.py:88: UserWarning: Could not fix matrix. Please try a different risk model.
  warnings.warn("Could not fix matrix. Please try a different risk model.")


OrderedDict([('a', 0.0), ('aa', 0.0), ('aaoi', 0.0), ('aapl', 0.0), ('aat', 0.0), ('aau', 0.0), ('abbv', 0.0), ('abcb', 0.0), ('abev', 0.0), ('abg', 0.0), ('abr', 0.0), ('abtx', 0.0), ('ac', 0.0), ('aca', 0.0), ('acad', 0.0), ('acb', 0.0), ('acc', 0.0), ('acel', 0.0), ('acet', 0.0), ('ach', 0.0), ('acia', 0.0), ('aciu', 0.0), ('acls', 0.0), ('acm', 0.0), ('acmr', 0.0), ('acn', 0.0), ('acor', 0.0), ('acp', 0.0), ('acre', 0.0), ('acrs', 0.01275), ('acrx', 0.0), ('acu', 0.0), ('acv', 0.0), ('adap', 0.0), ('adbe', 0.0), ('adil', 0.0), ('adms', 0.0), ('adnt', 0.0), ('ads', 0.0), ('adt', 0.0), ('adtn', 0.0), ('adus', 0.0), ('advm', 0.0), ('aehr', 0.0), ('aeis', 0.0), ('ael', 0.0), ('aenz', 0.0), ('aer', 0.0), ('aeri', 0.0), ('aese', 0.0), ('afb', 0.0), ('afi', 0.0), ('afmd', 0.0), ('aft', 0.0), ('agd', 0.0), ('age', 0.0), ('agen', 0.0), ('agfs', 0.0), ('agi', 0.0), ('agio', 0.0), ('agle', 0.0), ('agmh', 0.0), ('agnc', 0.0), ('ago', 0.0), ('agr', 0.0), ('agro', 0.0), ('agrx', 0.0), ('ags', 0.

(630886110045.1448, 8.246668334756227e-07, 7.650193804766053e+17)

In [ ]:
latest_prices = get_latest_prices(prices)
da = DiscreteAllocation(cleaned_weights, latest_prices, total_portfolio_value=6000)
allocation, leftover = da.lp_portfolio()

print(allocation)
print("Funds remaining: ${:.2f}".format(leftover))

In [ ]:
ef = EfficientFrontier(mu, S, weight_bounds=(-1, 1))
ef.efficient_return(target_return=0.2, market_neutral=True)